<h1 align="center">Data Visualization - Interactive Dashboard</h1>

<br>

<a id="0"></a> <br>
## Table of Contents
1. [Project Goal](#1)
2. [Average Number of Patients' Visits Per Hour](#2)
    1. [Visualization Design](#3)
    2. [Results Analysis](#4)
3. [Average CT Waiting Time & Demand Rate for TPA Per Hour](#5)
    1. [Visualization Design](#6)
    2. [Results Analysis](#7)
4. [Patients time spent in ICU and Neurology Ward](#8)
    1. [Visualization Design](#9)
    2. [Results Analysis](#10)
5. [Discover Connections Among Different Medical Specialists](#11)
    1. [Visualization Design](#12)
    2. [Results Analysis](#13)

<br>

<a id="1"></a>
## 1. Project Goal

- In this project, I designed four interactive visualizations for a hospital complex in Ontario to better monitor their processes around stroke care.

 - We design the dashboard in accordance with the patient's journey through the hospital system, from the time they arrive in the emergency department to the time they are discharged.


- First, patients arrive to hospital in the emergency department.


> (1) In this section, we focus on the average number of patients' visits per hour in a day. We use a polar chart the demonstrate the trends.


> (2) The main purpose of this visualization is to analyze the number of patients at different times, allowing the hospital to prepare in advance.


- Secondly, patients undergo a CT scan to diagnose the stroke, followed by intravenous thrombolysis (TPA) to break up the blood clot. This process is time-sensitive.


> (1) According to Canada Stroke Best Practices, quote, "Brain imaging (CT or MRI) and non-invasive vascular imaging (CTA or MRA) from aortic arch to vertex should be completed as soon as possible following acute disabling or non-disabling stroke, or TIA." (Refer to Section 2.2 (i).)


> (2) In this section, we analyze the average CT waiting time per hour for different patient groups (All patients, female patients, and male patients) to verify whether patients got a timely diagnosis.


> (3) Observed that about one-fifth of the patients need TPA, we also investigate the demand rate for TPA per hour for different patient groups (All patients, female patients, and male patients). Therefore, the hospital can prepare a corresponding number of TPA treatments.


- Thirdly, some patients may spend time in the intensive care unit (ICU). These patients need some special treatment.


> (1) In this section, we target the correlations between the time spent in the ICU and the neurology ward. We visualize the time distribution that patients spend in the ICU and the neurology ward.


> (2) The patients are grouped together by age, which is helpful to analyze the patient's demographic.


- Finally, patients are transferred to the neurology ward, where they are seen by a variety of medical specialists while they recover.


> (1) In this section, we are interested in connections among patients visited by different medical specialists. We give insights on the frequency of visits by different specialists, allowing the hospital to better coordinate the work of specialists.

In [1]:
# Load the Libraries
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import time
import math

In [3]:
# Define Constants
DATASET_NAME_STROKE = "dataset.csv"

In [4]:
# Load Data
df = pd.read_csv("./" + DATASET_NAME_STROKE)

<br>

<a id="2"></a>
## 2. Average Number of Patients' Visits Per Hour

In [5]:
df_emg_dept_time = df.copy()
# Get hourly data
df_emg_dept_time["Emergency Dept by Hour"] = pd.to_datetime(df_emg_dept_time["Emergency Dept Time"], yearfirst=True).dt.hour

In [6]:
# Group the data records by hour, and count the number of visits at each hour
df_radial = df_emg_dept_time.groupby(["Emergency Dept by Hour"])["Emergency Dept by Hour"].count().reset_index(name="count")

In [7]:
df_radial["clock"] = [1 for i in range(24)]

In [8]:
# Calculate angles for the polar coordinates
interval = (np.pi * 2) / df_radial["clock"].sum()

# Calculate the x-coordinates of each bin for the bar chart
value = np.cumsum([(i * interval) for i in df_radial["clock"].to_list()])
value = np.insert(value, 0, 0)[:-1]

# Calculate the height of each bin for the bar chart
height = df_radial["count"].to_list()

# Calculate the width of each bin for the bar chart
width = np.array([(i * interval) for i in df_radial["clock"].to_list()])

In [9]:
inner_color = []
for i in range(24):
    inner_color.append(["black" for j in range(i)] + ["red"] + ["black" for i in range(24 - i - 1)])

In [10]:
inner_text = []
for i in range(24):
    inner_text.append(df_radial.loc[i, "count"])    

In [11]:
# Calculate angles for the polar coordinates
angle = [i / 1440 * 2 * np.pi for i in range(1440)]

In [12]:
def f(play):
    # Define object-oriented interface, specify the projection as "polar"
    fig, ax = plt.subplots(figsize=(9, 9), subplot_kw=dict(projection="polar"))
    
    ax.bar(value, 
           height, 
           width, 
           linewidth=1, 
           color=inner_color[play % 24], 
           edgecolor="white", 
           align="edge")
    
    # Draw an angular axis at a radius of "252 per hour"
    sns.lineplot(x=angle, y=252, color="grey")

    # Draw an angular axis at a radius of "214 per hour" (the average value)
    sns.lineplot(x=angle, 
                 y = sum(height) / len(height), 
                 linestyle=(0, (2, 1)), 
                 linewidth=2, 
                 color="grey", 
                 zorder=10)
    
    # Draw dial plate
    for i in range(0, len(angle), 60):
        ax.scatter(angle[i], 252, s=17, zorder=10, color="grey");

    # Draw dial scale
    clock_text = ["Midnight", "1 A.M.", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", 
                  "Noon", "1 P.M.", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11"]
    j = -1
    for i in range(0, 360, 15):
        j += 1
        ax.text(np.radians(i), 270, clock_text[j], fontsize=12, ha='center', va='center')

    for i in range(24):
        if (play % 24 == i):
            ax.text(np.radians((play % 24) * 15 + 7), 
                    325, 
                    inner_text[play % 24], 
                    fontsize=15, 
                    weight="bold", 
                    color="black", 
                    ha="center", 
                    va="center")
        else:
            ax.text(np.radians((i % 24) * 15 + 7), 
                    310, 
                    inner_text[i % 24], 
                    fontsize=14, 
                    color="grey", 
                    ha="center", 
                    va="center")
    
    # Set title
    ax.set_title("Average Number of Patients' Visits Per Hour", weight="bold", fontsize=16, pad=30)
    
    ax.text(np.radians(40), 
            335, 
            "The average number\nof patients' visits\nper hour is " + str(round(sum(height) / len(height), 1)) + "\n(dashed line)", 
            fontsize=12)

    ax.text(np.radians(-35), 
            463, 
            "The outermost numbers represent the number of patients' visits in each hour", 
            fontsize=11)
    
    # Set x-axis tick labels
    ax.set_xticks([])
    ax.set_xlabel("")

    # Set y-axis tick labels
    ax.set_yticks([])
    ax.set_ylabel("")

    # Set radius limit
    ax.set_rlim(0, 365)
    
    # Set spines to invisible
    ax.spines["polar"].set_visible(False)

    # Set the location of theta's zero to "N"
    ax.set_theta_zero_location("N")

    # Set theta direction to clockwise
    ax.set_theta_direction(-1)

    # Get ready to save the figure
    fig.tight_layout()

# Define play button
play_widget = widgets.Play(
    value=0, 
    min=0, 
    max=8, 
    step=1, 
    interval=3700, 
    description="Press play", 
    disabled=False)

# Define output
out_play = widgets.interactive_output(f, {"play": play_widget})

# Set layout
widgets.VBox([play_widget, out_play])

<br>

<a id="3"></a>
### 2.1 Visualization Design

- In this section, we draw a polar chart to demonstrate the trends in a dial plate.

- We draw a dashed line at a radius of 214.5 representing the average number of patients' visits in a day.

- The outermost numbers represent the number of patients' visits in each hour.

- As we click the "play" button, our design works like a clock hand, turning clockwise and highlighting the corresponding bar and number.

<br>

<a id="4"></a>
### 2.2 Results Analysis

- We can see that the number of patients coming to the emergency department has remained stable overall. Therefore, the emergency department needs to be prepared at all times.

- At the same time, 9 pm to midnight is a small peak in the number of patients. Medical staff need to be alerted to any emergency.

<br>

<a id="5"></a>
## 3. Average CT Waiting Time & Demand Rate for TPA Per Hour

In [14]:
df_ct_time = df.copy()

In [15]:
# Get hourly data
df_ct_time["CT by Hour"] = pd.to_datetime(df_ct_time["CT Scan Time"], yearfirst=True).dt.hour
# Calculate CT waiting time
df_ct_time["CT Wait Time"] = (pd.to_datetime(df_ct_time["CT Scan Time"]) - pd.to_datetime(df_ct_time["Emergency Dept Time"])).dt.total_seconds()

In [16]:
# Get the average CT waiting time per hour, total patient number, and total TPA number
avg_ct_waiting_time = []
total_patients_number = []
total_tpa_number = []
for i in range(24):
    hourly_total_ct_waiting_time = 0
    hourly_patients_number = 0
    hourly_tpa_number = 0
    for j in range(len(df_ct_time)):
        if df_ct_time.loc[j, "CT by Hour"] == i:
            hourly_total_ct_waiting_time += df_ct_time.loc[j, "CT Wait Time"]
            hourly_patients_number += 1
        if (df_ct_time.loc[j, "CT by Hour"] == i) and (not pd.isna(df_ct_time.loc[j, "TPA Time"])):
            hourly_tpa_number += 1
            
    avg_ct_waiting_time.append(hourly_total_ct_waiting_time / hourly_patients_number)
    total_patients_number.append(hourly_patients_number)
    total_tpa_number.append(hourly_tpa_number)

In [17]:
# Get the average CT waiting time per hour, male patient number, and total TPA number for male patients
avg_ct_waiting_time_male = []
total_patients_number_male = []
total_tpa_number_male = []
for i in range(24):
    hourly_total_ct_waiting_time = 0
    hourly_patients_number = 0
    hourly_tpa_number = 0
    for j in range(len(df_ct_time)):
        if (df_ct_time.loc[j, "CT by Hour"] == i) and (df_ct_time.loc[j, "Gender"] == "M"):
            hourly_total_ct_waiting_time += df_ct_time.loc[j, "CT Wait Time"]
            hourly_patients_number += 1
        if (df_ct_time.loc[j, "CT by Hour"] == i) and (df_ct_time.loc[j, "Gender"] == "M") and (not pd.isna(df_ct_time.loc[j, "TPA Time"])):
            hourly_tpa_number += 1
    avg_ct_waiting_time_male.append(hourly_total_ct_waiting_time / hourly_patients_number)
    total_patients_number_male.append(hourly_patients_number)
    total_tpa_number_male.append(hourly_tpa_number)

In [18]:
# Get the average CT waiting time per hour, female patient number, and total TPA number for female patients
avg_ct_waiting_time_female = []
total_patients_number_female = []
total_tpa_number_female = []
for i in range(24):
    hourly_total_ct_waiting_time = 0
    hourly_patients_number = 0
    hourly_tpa_number = 0
    for j in range(len(df_ct_time)):
        if (df_ct_time.loc[j, "CT by Hour"] == i) and (df_ct_time.loc[j, "Gender"] == "F"):
            hourly_total_ct_waiting_time += df_ct_time.loc[j, "CT Wait Time"]
            hourly_patients_number += 1
        if (df_ct_time.loc[j, "CT by Hour"] == i) and (df_ct_time.loc[j, "Gender"] == "F") and (not pd.isna(df_ct_time.loc[j, "TPA Time"])):
            hourly_tpa_number += 1
    avg_ct_waiting_time_female.append(hourly_total_ct_waiting_time / hourly_patients_number)
    total_patients_number_female.append(hourly_patients_number)
    total_tpa_number_female.append(hourly_tpa_number)

In [19]:
df_ct_time_by_hour = df_ct_time.groupby(["CT by Hour"])["CT by Hour"].count().reset_index(name="count")

In [20]:
df_ct_time_by_hour["AVG CT Waiting Time"] = avg_ct_waiting_time
df_ct_time_by_hour["Count Male"] = total_patients_number_male
df_ct_time_by_hour["AVG CT Waiting Time Male"] = avg_ct_waiting_time_male
df_ct_time_by_hour["Count Female"] = total_patients_number_female
df_ct_time_by_hour["AVG CT Waiting Time Female"] = avg_ct_waiting_time_female

df_ct_time_by_hour["TPA Count"] = total_tpa_number
df_ct_time_by_hour["TPA Male Count"] = total_tpa_number_male
df_ct_time_by_hour["TPA Female Count"] = total_tpa_number_female

In [22]:
def f(gender, tpa):
    # Define object-oriented interface
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # The sizes of the points are proportional to the number of patients in different categories.
    if (gender == "All"):
        sns.scatterplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time"], 
            s=((df_ct_time_by_hour["count"] - df_ct_time_by_hour["count"].min()) / (df_ct_time_by_hour["count"].max() - df_ct_time_by_hour["count"].min()) * (21.45 - 1.0) + 1.0) * 90, 
            color="grey")
        sns.lineplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time"], 
            color="lightgrey", 
            linestyle=(0, (2, 1)), 
            linewidth=2, 
            zorder=-1)
        
        # Draw average line
        ax.axhline(y=round(df_ct_time_by_hour["AVG CT Waiting Time"].sum() / len(df_ct_time_by_hour["AVG CT Waiting Time"]), 2), 
                   xmax=24, 
                   color="dimgrey", 
                   linestyle="--", 
                   zorder = 3)

        ax.annotate("Average CT waiting time\nfor all patients\nin a day: 14m 59s", 
                    color="dimgrey", 
                    xy=(-0.5, 900), 
                    xycoords="data", 
                    xytext=(2, 820), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=-90", color="dimgrey"))
    
    elif (gender == "Female"):
        sns.scatterplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Female"], 
            s=((df_ct_time_by_hour["Count Female"] - df_ct_time_by_hour["Count Female"].min()) / (df_ct_time_by_hour["Count Female"].max() - df_ct_time_by_hour["Count Female"].min()) * (10.0 - 1.0) + 1.0) * 90, 
            color="mediumturquoise")
        sns.lineplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Female"], 
            color="lightgrey", 
            linestyle=(0, (2, 1)), 
            linewidth=2, 
            zorder=-1)
        
        # Draw average line
        ax.axhline(y=round(df_ct_time_by_hour["AVG CT Waiting Time Female"].sum() / len(df_ct_time_by_hour["AVG CT Waiting Time Female"]), 2), 
                   xmax=24, 
                   color="dimgrey", 
                   linestyle="--", 
                   zorder = 3)
    
        ax.annotate("Average CT waiting time\nfor female patients\nin a day: 14m 59s", 
                    color="dimgrey", 
                    xy=(-0.5, 900), 
                    xycoords="data", 
                    xytext=(2, 820), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=-90", color="dimgrey"))
    
    elif (gender == "Male"):
        sns.scatterplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Male"], 
            s=((df_ct_time_by_hour["Count Male"] - df_ct_time_by_hour["Count Male"].min()) / (df_ct_time_by_hour["Count Male"].max() - df_ct_time_by_hour["Count Male"].min()) * (15.32 - 1.0) + 1.0) * 90, 
            color="darkorange")
        sns.lineplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Male"], 
            color="lightgrey", 
            linestyle=(0, (2, 1)), 
            linewidth=2, 
            zorder=-1)
        
        # Draw average line
        ax.axhline(y=round(df_ct_time_by_hour["AVG CT Waiting Time Male"].sum() / len(df_ct_time_by_hour["AVG CT Waiting Time Male"]), 2), 
                   xmax=24, 
                   color="dimgrey", 
                   linestyle="--", 
                   zorder = 3)

        ax.annotate("Average CT waiting time\nfor male patients\nin a day: 14m 59s", 
                    color="dimgrey", 
                    xy=(-0.5, 900), 
                    xycoords="data", 
                    xytext=(2, 820), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=-90", color="dimgrey"))
    
    elif (gender == "Compare"):
        sns.scatterplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Male"], 
            s=((df_ct_time_by_hour["Count Male"] - df_ct_time_by_hour["Count Male"].min()) / (df_ct_time_by_hour["Count Male"].max() - df_ct_time_by_hour["Count Male"].min()) * (15.32 - 1.0) + 1.0) * 90, 
            color="darkorange")
        sns.lineplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Male"], 
            color="darkgrey", 
            linestyle=(0, (2, 1)), 
            linewidth=2, 
            zorder=-1)
    
        sns.scatterplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Female"], 
            s=((df_ct_time_by_hour["Count Female"] - df_ct_time_by_hour["Count Female"].min()) / (df_ct_time_by_hour["Count Female"].max() - df_ct_time_by_hour["Count Female"].min()) * (10.0 - 1.0) + 1.0) * 90, 
            color="mediumturquoise")
        sns.lineplot(
            data=df_ct_time_by_hour, 
            x=df_ct_time_by_hour["CT by Hour"], 
            y=df_ct_time_by_hour["AVG CT Waiting Time Female"], 
            color="lightgrey", 
            linestyle=(0, (2, 1)), 
            linewidth=2, 
            zorder=-1)

        # Draw average line
        ax.axhline(y=round(df_ct_time_by_hour["AVG CT Waiting Time"].sum() / len(df_ct_time_by_hour["AVG CT Waiting Time"]), 2), 
                   xmax=24, 
                   color="dimgrey", 
                   linestyle="--", 
                   zorder = 3)
        
        ax.annotate("Female", 
                    color="mediumturquoise", 
                    xy=(20, 924), 
                    xycoords="data", 
                    xytext=(21.5, 939), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=90", color="mediumturquoise"))

        ax.annotate("Male", 
                    color="darkorange", 
                    xy=(21, 895), 
                    xycoords="data", 
                    xytext=(22.5, 880), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=-90", color="darkorange"))
    
        ax.annotate("Average CT waiting time\nfor all patients\nin a day: 14m 59s", 
                    color="dimgrey", 
                    xy=(-0.5, 900), 
                    xycoords="data", 
                    xytext=(2, 820), 
                    textcoords="data", 
                    arrowprops=dict(arrowstyle="->", connectionstyle="angle3, angleA=0, angleB=-90", color="dimgrey"))
    
    if (tpa and (gender == "All")):
        for i in range(24):
            ax.text(i - 0.5, 
                    980, 
                    str(round((df_ct_time_by_hour.loc[i, "TPA Count"] * 10000) / (df_ct_time_by_hour.loc[i, "count"] * 100), 2)) + "%", 
                    rotation=45)
            
    if (tpa and (gender == "Compare")):
        for i in range(24):
            ax.text(i - 0.5, 
                    980, 
                    str(round((df_ct_time_by_hour.loc[i, "TPA Count"] * 10000) / (df_ct_time_by_hour.loc[i, "count"] * 100), 2)) + "%", 
                    rotation=45)

    if (tpa and (gender == "Female")):
        for i in range(24):
            ax.text(i - 0.5, 
                    980, 
                    str(round((df_ct_time_by_hour.loc[i, "TPA Female Count"] * 10000) / (df_ct_time_by_hour.loc[i, "Count Female"] * 100), 2)) + "%", 
                    rotation=45)

    if (tpa and (gender == "Male")):
        for i in range(24):
            ax.text(i - 0.5, 
                    980, 
                    str(round((df_ct_time_by_hour.loc[i, "TPA Male Count"] * 10000) / (df_ct_time_by_hour.loc[i, "Count Male"] * 100), 2)) + "%", 
                    rotation=45)
    
    # Set the color of spines
    ax.spines['bottom'].set_color("lightgrey")
    ax.spines['top'].set_color("lightgrey") 
    ax.spines['right'].set_color("lightgrey")
    ax.spines['left'].set_color("lightgrey")

    ax.tick_params(left=False, bottom=False)
    
    # Set x-tick marks and labels
    x_ticks = [i for i in range(24)]
    ax.set_xticks(x_ticks)
    # Set specified x-ticks
    x_tick_labels = [i for i in range(24)]
    ax.set_xticklabels(x_tick_labels)
    
    # Set y-tick marks and labels
    y_ticks = [800, 825, 850, 875, 900, 925, 950, 975, 1000]
    ax.set_yticks(y_ticks)
    # Set specified y-ticks
    y_tick_labels = ["13m 20s", "13m 45s", "14m 10s", "14m 35s", "15m 00s", "15m 25s", "15m 50s", "16m 15s", "16m 40s"]
    ax.set_yticklabels(y_tick_labels)
    
    # Set x-axes limits
    ax.set_xlim(-1, 24)
    # Set x-axes label
    ax.set_xlabel("Hours in a Day")
    # Set x-axes label padding
    ax.xaxis.labelpad = 20

    # Set y-axes limits
    ax.set_ylim(800, 1000)
    # Set y-axes label
    ax.set_ylabel("Average CT Waiting Time")
    
    # Set title
    ax.set_title("Average CT Waiting Time & Demand Rate for TPA Per Hour", weight="bold", fontsize=16, pad=50)

    ax.text(2.5, 
            1013, 
            "The size of the point represents numerical relationships among the number of visits per hour", 
            fontsize=12)
    
    ax.text(5, 
            785, 
            "Morning")

    ax.text(17, 
            785, 
            "Evening")
    
    # Get ready to save the figure
    fig.tight_layout()
    
# Define widgets
gender_widget = widgets.ToggleButtons(options=["Female", "Male", "Compare", "All"], 
                                      description="Category:", 
                                      button_style="info")

tpa_widget = widgets.Checkbox(value=False, description="TPA")

# Set layout
ui = widgets.HBox([gender_widget, tpa_widget])

# Define output
out = widgets.interactive_output(f, 
                                 {"gender": gender_widget, 
                                  "tpa": tpa_widget
                                 })

# Set layout
widgets.VBox([ui, out])

<br>

<a id="6"></a>
### 3.1 Visualization Design

- In the scatter plot, we show the average CT waiting time per hour.

- We can also check the average CT waiting time for different patient groups by selecting the different toggle buttons.

- The size of the point throughout the visualization represents numerical relationships among the number of visits per hour.

- The "compare" selection uses the same data as the "All" selection, just divided into female and male groups. Therefore, they represent the same number of patients. We can observe this from the sizes of the points.

- We draw the average CT waiting time in a day using a horizontal dashed line.

- We also draw some annotations to aid the interpretation.

- By selecting the TPA checkbox, we can see the demand rate for TPA per hour on the top of the scatter plot for different patient groups.

<br>

<a id="7"></a>
### 3.2 Results Analysis

- As we can see, the average CT waiting times vary little, although there is a delay of no more than 1 minute when the number of patients is large.

- There was no significant difference in CT waiting time between female and male patients.

- At the same time, the average CT waiting time for different groups of patients is the same (14m 59s). This reflects the consistency of the hospital's work.

- Furthermore, about one in ten patients need TPA treatment. 

- Female patients require TPA at a slightly higher rate than male patients.

<br>

<a id="8"></a>
## 4. Patients time spent in ICU and Neurology Ward

In [23]:
df_time = df.copy()

# Calculate different periods
df_time["ICU Period"] = (pd.to_datetime(df_time["ICU Checkout Time"]) - pd.to_datetime(df_time["ICU Arrival Time"])).dt.total_seconds()
df_time["Neurology Ward Period"] = (pd.to_datetime(df_time["Discharge Time"]) - pd.to_datetime(df_time["Neurology Ward Arrival Time"])).dt.total_seconds()

In [24]:
# We only analyze on the patinents who have been in ICU
df_time = df_time.dropna(subset=["ICU Period"])

In [25]:
def f(Age, Comorbidities):    
    # Define object-oriented interface
    fig, ax = plt.subplots(figsize=(6.5, 6.5))
    
    if (Comorbidities == "All"):
        sns.histplot(data=df_time, x=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5)]["ICU Period"], y=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5)]["Neurology Ward Period"], stat="count", binwidth=375000,  binrange=((0, 6e6), (0, 6e6)), cbar=False, cmap="Reds")
    
    elif (Comorbidities == "With Comorbidities"):
        sns.histplot(data=df_time, x=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5) & (df_time["Comorbidities"] == True)]["ICU Period"], y=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5) & (df_time["Comorbidities"] == True)]["Neurology Ward Period"], stat="count", binwidth=375000,  binrange=((0, 6e6), (0, 6e6)), cbar=False, cmap="Reds")
    
    elif (Comorbidities == "Without Comorbidities"):
        sns.histplot(data=df_time, x=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5) & (df_time["Comorbidities"] == False)]["ICU Period"], y=df_time[(df_time["Age"] >= Age) & (df_time["Age"] <= Age + 5) & (df_time["Comorbidities"] == False)]["Neurology Ward Period"], stat="count", binwidth=375000,  binrange=((0, 6e6), (0, 6e6)), cbar=False, cmap="Reds")
        
    # Draw grid
    lines_count = 0
    while (375000 * (lines_count + 1) <= 6000000):
        ax.axvline(x=375000 * (lines_count + 1), ymax=6000000, color="whitesmoke")
        ax.axhline(y=375000 * (lines_count + 1), xmax=6000000, color="whitesmoke")
        lines_count += 1
    
    # Draw text box
    ax.text(2500000, 
            5000000, 
            "Age: " + str(Age) + " to " + str(Age + 5), 
            fontsize=25, 
            weight="bold", 
            bbox=dict(facecolor="whitesmoke", 
                      edgecolor="gainsboro", 
                      boxstyle="round", 
                      linewidth=3)
           )
    
    # Set x-axes limits
    ax.set_xlim(0, 6e6)
    # Set x-axes label
    ax.set_xlabel("ICU Period (days)")

    # Set y-axes limits
    ax.set_ylim(0, 6e6)
    # Set y-axes label
    ax.set_ylabel("Neurology Ward Period (days)")
    
    # Set right and top spines to invisible
    ax.spines[["right", "top"]].set_visible(False)
    
    # Set x-tick marks
    x_ticks = [1 * 24 * 60 * 60, 5 * 24 * 60 * 60, 10 * 24 * 60 * 60, 20 * 24 * 60 * 60, 30 * 24 * 60 * 60, 40 * 24 * 60 * 60, 50 * 24 * 60 * 60, 60 * 24 * 60 * 60, 65 * 24 * 60 * 60, 6000000]
    ax.set_xticks(x_ticks)
    # Set specified x-ticks
    x_tick_labels = ["1d", "5d", "10d", "20d", "30d", "40d", "50d", "60d", "65d", ""]
    ax.set_xticklabels(x_tick_labels, rotation=45)
    
    # Set y-tick marks
    y_ticks = [1 * 24 * 60 * 60, 5 * 24 * 60 * 60, 10 * 24 * 60 * 60, 20 * 24 * 60 * 60, 30 * 24 * 60 * 60, 40 * 24 * 60 * 60, 50 * 24 * 60 * 60, 60 * 24 * 60 * 60, 65 * 24 * 60 * 60, 6000000]
    ax.set_yticks(y_ticks)
    # Set specified x-ticks
    y_tick_labels = ["1d", "5d", "10d", "20d", "30d", "40d", "50d", "60d", "65d", ""]
    ax.set_yticklabels(y_tick_labels)
    
    # Set title
    ax.set_title("Patients time spent in ICU and Neurology Ward", weight="bold", fontsize=16, pad=20)

    # Get ready to save the figure
    fig.tight_layout()


# Define widgets
age_widget = widgets.IntSlider(value=60, min=30,max=90,step=5, continuous_update=True, style={"handle_color": "tomato"}, description="Age:")

comorbidities_widget = widgets.ToggleButtons(options=["All", "With Comorbidities", "Without Comorbidities"], 
                                             description="Category:", 
                                             button_style="info", 
                                             style={"button_width": "180px"})

# Set layout
ui = widgets.VBox([age_widget, comorbidities_widget])

# Define output
out = widgets.interactive_output(f, 
                                 {
                                     "Age": age_widget, 
                                     "Comorbidities": comorbidities_widget
                                 })

# Set layout
widgets.VBox([out, ui])    

<br>

<a id="9"></a>
### 4.1 Visualization Design

- In this section, we draw a heatmap that shows the correlations between patients' time spent in the ICU and neurology ward.

- The darker the color, the more patients there are.

- We grouped patients with an age difference of 5 as a unit.

- We can select different age groups through the slider bar. Also, we highlight the selected age group in the upper right corner of the heatmap.

- By selecting different toggle buttons, we can check the time correlations in the case of the presence or absence of comorbidities, which helps analyze the patient's demographic.

- In addition, the x and y-axes of the heatmap are marked with inequality to facilitate our interpretation.

<br>

<a id="10"></a>
### 4.2 Results Analysis

- As we can see, more patients spent less than 5 days in the ICU and the neurology ward.

- For other patients, time spent in the ICU is inversely proportional to time spent in the neurology ward.

- This potential correlation held for all age groups.

- Furthermore, patients with comorbidities spent more time in ICU than those without comorbidities. At the same time, patients with comorbidities spent significantly more time in the neurology ward.

- And this potential correlation held for all age groups.

- Through this visualization, the hospital may better arrange wards for patients with different conditions.

<br>

<a id="11"></a>
## 5. Discover Connections Among Different Medical Specialists

In [26]:
df_experts_visit = df[["Occupational Therapist Visit", "Speech Pathologist Visit", "Physiotherapist Visit", "Dietitian Visit", "Social Worker Visit" ,"Cardiologist Visit", "Neurologist Visit"]]

In [27]:
experts_list = [
    "Neurologist Visit",
    "Physiotherapist Visit",
    "Social Worker Visit",
    "Speech Pathologist Visit",
    "Occupational Therapist Visit",
    "Dietitian Visit",
    "Cardiologist Visit"
]

In [28]:
experts_name_list = [
    "Neurologist",
    "Physiotherapist",
    "Social Worker",
    "Speech Pathologist",
    "Occupational Therapist",
    "Dietitian",
    "Cardiologist"
]

In [30]:
total_concurrence_count = np.zeros((7, 7))

In [31]:
total_concurrence_count_norm = np.zeros((7, 7))

In [32]:
# Calculate the number of visits to the same patient by different specialists
min_max_finder = []
for i in range(0, len(experts_list) - 1):
    for j in range(i + 1, len(experts_list)):
        concurrence_count = 0
    
        for k in range(len(df_experts_visit)):
            if ((not pd.isna(df_experts_visit.loc[k, experts_list[i]])) and (not pd.isna(df_experts_visit.loc[k, experts_list[j]]))):
                concurrence_count += 1
        min_max_finder.append(concurrence_count)

for i in range(0, len(experts_list) - 1):
    for j in range(i + 1, len(experts_list)):
        concurrence_count = 0
    
        for k in range(len(df_experts_visit)):
            if ((not pd.isna(df_experts_visit.loc[k, experts_list[i]])) and (not pd.isna(df_experts_visit.loc[k, experts_list[j]]))):
                concurrence_count += 1
        total_concurrence_count[i][j] = concurrence_count
        total_concurrence_count[j][i] = concurrence_count

total_concurrence_count = np.array(total_concurrence_count, dtype=int)

for i in range(0, len(experts_list) - 1):
    for j in range(i + 1, len(experts_list)):
        concurrence_count = 0
    
        for k in range(len(df_experts_visit)):
            if ((not pd.isna(df_experts_visit.loc[k, experts_list[i]])) and (not pd.isna(df_experts_visit.loc[k, experts_list[j]]))):
                concurrence_count += 1
        total_concurrence_count_norm[i][j] = (concurrence_count - min(min_max_finder)) / (max(min_max_finder) - min(min_max_finder)) * (1.0 - 0.1) + 0.1
        total_concurrence_count_norm[j][i] = (concurrence_count - min(min_max_finder)) / (max(min_max_finder) - min(min_max_finder)) * (1.0 - 0.1) + 0.1

In [33]:
def f(xaxis, yaxis):
    # Define object-oriented interface
    fig, ax = plt.subplots(figsize=(6.5, 6.5))

    cm = sns.color_palette("YlOrBr", as_cmap = True)
    
    bold_index_x = -100
    bold_index_y = -100
    
    for i in range(len(experts_name_list)):
        if experts_name_list[i] == xaxis:
            bold_index_x = i
            
    experts_name_list.reverse()
    for i in range(len(experts_name_list)):
        if experts_name_list[i] == yaxis:
            bold_index_y = i
    experts_name_list.reverse()

    for i in range(7):
        for j in range(7):
            if 6 - i != j:
                if (i == bold_index_x) and (j == bold_index_y):
                    sns.scatterplot(x=[i], y=[j], s=1150, edgecolor="black", facecolor=cm(total_concurrence_count_norm[6 - j, i]))
                else:
                    sns.scatterplot(x=[i], y=[j], s=1150, color=cm(total_concurrence_count_norm[6 - j, i]))

    for i in range(len(experts_name_list)):
        if experts_name_list[i] == xaxis:
            fill_x = [j for j in range(i + 1)]
            fill_x = [-0.4] + fill_x + [i + 0.4]
    
    experts_name_list.reverse()
    for i in range(len(experts_name_list)):
        if experts_name_list[i] == yaxis:
            fill_y1 = i - 0.4
            fill_y2 = i + 0.4
    experts_name_list.reverse()
    
    ax.fill_between(x = fill_x, y1 = fill_y1, y2 = fill_y2, color="gainsboro", zorder = -1)
    
    
    for i in range(len(experts_name_list)):
        if experts_name_list[i] == xaxis:
            fill_x = [i - 0.4, i + 0.4]
    
    experts_name_list.reverse()
    for i in range(len(experts_name_list)):
        if experts_name_list[i] == yaxis:
            fill_y1 = -0.4
            fill_y2 = i + 0.4
    experts_name_list.reverse()
    
    ax.fill_between(x = fill_x, y1 = fill_y1, y2 = fill_y2, color="gainsboro", zorder = -1)

    # Set y-tick marks
    ax.set_yticks([i for i in range(7)])
    # Set specified y-ticks
    experts_name_list.reverse()
    ax.set_yticklabels(experts_name_list)

    # Set x-tick marks
    ax.set_xticks([i for i in range(7)])
    # Set specified x-ticks
    experts_name_list.reverse()
    ax.set_xticklabels(experts_name_list, rotation=60)
    
    # Set the current selected labels to bold
    for temp_label in ax.get_xticklabels():
        if temp_label.get_text() == xaxis:
            temp_label.set_fontweight("bold")
    
    for temp_label in ax.get_yticklabels():
        if temp_label.get_text() == yaxis:
            temp_label.set_fontweight("bold")

    # Set x-axis tick markers to invisible
    ax.tick_params(bottom = False)
    # Set y-axis tick markers to invisible
    ax.tick_params(left = False)

    # Set left, right, top, and bottom spines to invisible
    ax.spines[["left", "right", "top", "bottom"]].set_visible(False)

    ax.set_xlim(-1, 7)
    ax.set_ylim(-1, 7)
    
    # Set plot title
    ax.set_title("Discover Connections Among Differnt Specialists", weight="bold", fontsize=16, pad=20)

    # Get ready to save the figure
    fig.tight_layout()

# Define widgets
xaxis_widget = widgets.Dropdown(options=experts_name_list, 
                                value="Occupational Therapist", 
                                description="Specialist 1:")
yaxis_widget = widgets.Dropdown(options=experts_name_list, 
                                value="Social Worker", 
                                description="Specialist 2:")


record_concurrence_index_x = -100
record_concurrence_index_y = -100

for i in range(len(experts_name_list)):
    if (experts_name_list[i] == xaxis_widget.value):
        record_concurrence_index_x = i
    if (experts_name_list[i] == yaxis_widget.value):
        record_concurrence_index_y = i

# Define output text box (use the text box to display the detailed information)
output_text = widgets.Textarea(
    value=xaxis_widget.value + " and " + yaxis_widget.value + " checked on a patient together [" + str(total_concurrence_count[record_concurrence_index_x][record_concurrence_index_y]) + "] times.",
    description="Description:",
    disabled=False,
    layout=widgets.Layout(height="50%", width="auto")
    
)

# Calculate the number of visits to the same patient by two specified specialists
def update_output_text(*args):
    record_concurrence_index_x = -100
    record_concurrence_index_y = -100

    for i in range(len(experts_name_list)):
        if (experts_name_list[i] == xaxis_widget.value):
            record_concurrence_index_x = i
        if (experts_name_list[i] == yaxis_widget.value):
            record_concurrence_index_y = i

    output_text.value = xaxis_widget.value + " and " + yaxis_widget.value + " checked on a patient together [" + str(total_concurrence_count[record_concurrence_index_x][record_concurrence_index_y]) + "] times."

# Create call back function to listen to the events and update the text box
xaxis_widget.observe(update_output_text, "value")
yaxis_widget.observe(update_output_text, "value")

# Set layout
ui = widgets.VBox([xaxis_widget, yaxis_widget, output_text])

# Define output
out = widgets.interactive_output(f, {"xaxis": xaxis_widget, 
                                     "yaxis": yaxis_widget})

# Set layout
widgets.HBox([out, ui])

<br>

<a id="12"></a>
### 5.1 Visualization Design

- In this section, we visualize connections among different medical specialists.

- We can select two specialists we are interested in from two dropdown menus in the right operation panel.

- After the selections, the output displayed on the left will highlight the point that we selected with a black edge.

- We also highlight the selected specialists in bold in the x and y-axis labels and mark the path from the axis to the selected point with grey shadows.

- The color of the points corresponds to the number of times both specialists have visited the patient. The darker the color, the more concurrence there is.

- Furthermore, we use the textarea as an output displayer. Textarea is updated when triggered by dropdown menus.

- It shows the number of times two selected specialists checked on a patient together.

<br>

<a id="13"></a>
### 5.2 Results Analysis

- As we can see, the frequency of patient visits by different specialists is shown in the sequence from left to right on the x-axis in the visualization.

- We can see not only how often different specialists visit patients, but also the specific number of times two specialists visit patients.

- The hospital can take advantage of this result by rationalizing the number and working hours of different specialists.